# Data Description

I have chosen [ERA5-land hourly data](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form) for my project. 
The objective is to find useful information about the weather in the Konkan area of Maharashtra, which receives the highest rainfall, leading to the floods. If we are able to find useful patterns in the data that could help us forecast the precipitation in the future, even by an hour, it could help the people to better prepare for the flood. 

- The data is in `netCDF` file format. Hence, we can use `xarray` to read the data.

**Floods in Mahad**
- My hometown is Mahad, a small town in the Konkan district of Maharashtra, India. 
- On July 21 and July 22, 2021, the torrential rains had left `94` houses completely damaged and `9649` partially wrecked.
- About `3709` houses were damaged. 
- The landslide in **Taliye** village in the Mahad tehshil killed at least 84 people.

**The parameters that I have chosen are:**
- `10m_v_component_of_wind` or `v10` : northward component of the 10m wind, in `m/s` units, moving at a height of 10m above the surface,
- `2m_temperature` or `t2m`: the temperature of air at 2m above the surface, in `Kelvin` units,
- `surface_net_solar_radiation` or `ssr`: the amount of solar radiation that reaches a horizontal plane at the surface minus the amount reflected by the Earth's surface, in `W/m^2` units,
- `surface_pressure` or `sp`: pressue of the atmosphere on the surface of land, in `Pa` units,
- `total_precipitation` or `tp`: the accumulated liquid and frozen water, comprising rain and snow, that falls to the surface in `m` units.

**Further information about the floods in Mahad and the havoc it caused**
- https://www.business-standard.com/article/current-affairs/maha-over-9-700-houses-45-buildings-damaged-in-mahad-due-to-july-floods-121081301002_1.html#:~:text=A%20total%20of%2045%20buildings,occurred%20on%20July%2021%2D22. 



# Install required libraries and [Contourf and cartopy issues resolved](https://linuxtut.com/en/8f0d69169dc6ab753e08/)

In [1]:
!pip install xarray
!pip install cartopy

!grep '^deb ' /etc/apt/sources.list | \
  sed 's/^deb /deb-src /g' | \
  tee /etc/apt/sources.list.d/deb-src.list
!apt-get -qq update

!grep '^deb ' /etc/apt/sources.list | \
  sed 's/^deb /deb-src /g' | \
  tee /etc/apt/sources.list.d/deb-src.list
!apt-get update

!apt-get -qq build-dep python3-cartopy
!apt-get -qq remove python-shapely python3-shapely

!pip install --no-binary shapely shapely --force
!pip install --no-binary cartopy cartopy==0.19.0

!pip list | grep Shapely

     |████████████████████████████████| 10.8 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 36.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 10.8 MB 31.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
     |████████████████████████████████| 12.1 MB 30.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 44 kB 2.1 MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516284 sha256=79b5a89953a63f0db13fe2a18d086d10de066a3a5619f997540beeb7bcdb467b
  Stored in directory: /root/.cache/pip/wheels/98/01/f7/bd10aeb96fe4b518cde5f7c4f5e12c7202f85b7353a5017847
Successfully built cartopy
deb-src h

# Install libraries

In [2]:
import numpy as np  # import numpy package 
import matplotlib.pyplot as plt # function in matplotlib that produces quick and easy plots
import pandas as pd # load the Pandas module, which is a library for data manipulation of 2 dimensional data
import xarray as xr # load the xarray module, which is a library for data manipulation of N dimensional data
import cartopy.crs as ccrs # Cartopy map projections
import cartopy.feature as cfeature # Cartopy map features


# Load data

* The data 'download.nc' had been downloaded using the 'data_collection.ipynb' file. I kept it as a separate file, because the website might keep changing the format of the data, time to time. I want the results to be consistent.


In [3]:
# open_dataset is one of the main functions in Xarray to open a Netcdf file.  
data = xr.open_dataset('download.nc') # open the dataset
data 

<xarray.Dataset>
Dimensions:    (latitude: 61, longitude: 71, time: 792)
Coordinates:
  * longitude  (longitude) float32 72.0 72.1 72.2 72.3 ... 78.7 78.8 78.9 79.0
  * latitude   (latitude) float32 22.0 21.9 21.8 21.7 ... 16.3 16.2 16.1 16.0
  * time       (time) datetime64[ns] 2019-07-14 ... 2021-07-24T23:00:00
Data variables:
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    ssr        (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-03-23 20:02:03 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

# Let's evaluate variables and coordinates - always a good first step



In [4]:
# What are the dimensions of the five data variables - specifically, the named coordinates associated with them?
print(f"The dimensions of the v10 variable are: {data.v10.dims}")
print(f"The dimensions of the t2m variable are: {data.t2m.dims}")
print(f"The dimensions of the ssr variable are: {data.ssr.dims}")
print(f"The dimensions of the sp variable are: {data.sp.dims}")
print(f"The dimensions of the tp variable are: {data.tp.dims}")

The dimensions of the v10 variable are: ('time', 'latitude', 'longitude')
The dimensions of the t2m variable are: ('time', 'latitude', 'longitude')
The dimensions of the ssr variable are: ('time', 'latitude', 'longitude')
The dimensions of the sp variable are: ('time', 'latitude', 'longitude')
The dimensions of the tp variable are: ('time', 'latitude', 'longitude')


In [5]:
# coordinate variables
print(data.coords)

Coordinates:
  * longitude  (longitude) float32 72.0 72.1 72.2 72.3 ... 78.7 78.8 78.9 79.0
  * latitude   (latitude) float32 22.0 21.9 21.8 21.7 ... 16.3 16.2 16.1 16.0
  * time       (time) datetime64[ns] 2019-07-14 ... 2021-07-24T23:00:00


In [8]:
# Is the longitude coordinates in the 0 to 360 E system, or the 180 W to 180 E system?
print(data['longitude'])
print('\n')
print(data['latitude'])
# a description of the direction of the order of each of the spatial coordinates (northward? southward? eastward? westward?)
print(f"The longitude coordinate is eastward")
print(f"The latitude coordinate is northward")

<xarray.DataArray 'longitude' (longitude: 71)>
array([72. , 72.1, 72.2, 72.3, 72.4, 72.5, 72.6, 72.7, 72.8, 72.9, 73. , 73.1,
       73.2, 73.3, 73.4, 73.5, 73.6, 73.7, 73.8, 73.9, 74. , 74.1, 74.2, 74.3,
       74.4, 74.5, 74.6, 74.7, 74.8, 74.9, 75. , 75.1, 75.2, 75.3, 75.4, 75.5,
       75.6, 75.7, 75.8, 75.9, 76. , 76.1, 76.2, 76.3, 76.4, 76.5, 76.6, 76.7,
       76.8, 76.9, 77. , 77.1, 77.2, 77.3, 77.4, 77.5, 77.6, 77.7, 77.8, 77.9,
       78. , 78.1, 78.2, 78.3, 78.4, 78.5, 78.6, 78.7, 78.8, 78.9, 79. ],
      dtype=float32)
Coordinates:
  * longitude  (longitude) float32 72.0 72.1 72.2 72.3 ... 78.7 78.8 78.9 79.0
Attributes:
    units:      degrees_east
    long_name:  longitude


<xarray.DataArray 'latitude' (latitude: 61)>
array([22. , 21.9, 21.8, 21.7, 21.6, 21.5, 21.4, 21.3, 21.2, 21.1, 21. , 20.9,
       20.8, 20.7, 20.6, 20.5, 20.4, 20.3, 20.2, 20.1, 20. , 19.9, 19.8, 19.7,
       19.6, 19.5, 19.4, 19.3, 19.2, 19.1, 19. , 18.9, 18.8, 18.7, 18.6, 18.5,
       18.4, 18.3, 1